# Required Libraries

In [2]:
import pandas as pd
import numpy as np

import random

# Set the random seed for the random module
random.seed(42)

# Set the random seed for NumPy
np.random.seed(42)

# Datasets

Data retrieved from: https://www.data.qld.gov.au/dataset/crash-data-from-queensland-roads/resource/18ee2911-992f-40ed-b6ae-e756859786e6?inner_span=True

There are 4 crash datasets to be preprocessed. They involve:
1. Road crash locations
2. Driver demographics
3. Vehicle types
4. Road crash factors

These 4 were chosen as they share multiple columns which will allow for joining. The road casualties and seatbelt restraints datasets were excluded as they did not share columns.

In [3]:
df_crash_locations = pd.read_csv("crash_data_queensland_1_crash_locations.csv")
df_crash_driver_demographics = pd.read_csv("crash_data_queensland_b_driver_involvement.csv")
df_crash_vehicle_types = pd.read_csv("crash_data_queensland_d_vehicle_involvement.csv")
df_crash_factors = pd.read_csv("crash_data_queensland_e_alcohol_speed_fatigue_defect.csv")

Property damage crashes were recorded only up to 2010, therefore crashes that resulted in these will be excluded

In [4]:
df_locations = df_crash_locations[df_crash_locations['Crash_Severity'] != 'Property damage only']
df_driver_demographics = df_crash_driver_demographics[df_crash_driver_demographics['Crash_Severity'] != 'Property damage only']
df_vehicle_types = df_crash_vehicle_types[df_crash_vehicle_types["Crash_Severity"] != "Property damage"]
df_factors = df_crash_factors[df_crash_factors["Crash_Severity"] != "Property damage"]

Now to get an initial look at the four datasets

In [5]:
df_locations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279798 entries, 0 to 367229
Data columns (total 52 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Crash_Ref_Number                 279798 non-null  int64  
 1   Crash_Severity                   279798 non-null  object 
 2   Crash_Year                       279798 non-null  int64  
 3   Crash_Month                      279798 non-null  object 
 4   Crash_Day_Of_Week                279798 non-null  object 
 5   Crash_Hour                       279798 non-null  int64  
 6   Crash_Nature                     279798 non-null  object 
 7   Crash_Type                       279798 non-null  object 
 8   Crash_Longitude                  279798 non-null  float64
 9   Crash_Latitude                   279798 non-null  float64
 10  Crash_Street                     279791 non-null  object 
 11  Crash_Street_Intersecting        121005 non-null  object 
 12  St

In [6]:
df_driver_demographics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 19895
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Crash_Year                          16713 non-null  int64 
 1   Crash_Police_Region                 16713 non-null  object
 2   Crash_Severity                      16713 non-null  object
 3   Involving_Male_Driver               16713 non-null  object
 4   Involving_Female_Driver             16713 non-null  object
 5   Involving_Young_Driver_16-24        16713 non-null  object
 6   Involving_Senior_Driver_60plus      16713 non-null  object
 7   Involving_Provisional_Driver        16713 non-null  object
 8   Involving_Overseas_Licensed_Driver  16713 non-null  object
 9   Involving_Unlicensed_Driver         16713 non-null  object
 10  Count_Crashes                       16713 non-null  int64 
 11  Count_Casualty_Fatality             16713 non-null  in

In [7]:
df_vehicle_types.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3246 entries, 0 to 3245
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Crash_Year                       3246 non-null   int64 
 1   Crash_Police_Region              3246 non-null   object
 2   Crash_Severity                   3246 non-null   object
 3   Involving_Motorcycle_Moped       3246 non-null   object
 4   Involving_Truck                  3246 non-null   object
 5   Involving_Bus                    3246 non-null   object
 6   Count_Crashes                    3246 non-null   int64 
 7   Count_Casualty_Fatality          3246 non-null   int64 
 8   Count_Casualty_Hospitalised      3246 non-null   int64 
 9   Count_Casualty_MedicallyTreated  3246 non-null   int64 
 10  Count_Casualty_MinorInjury       3246 non-null   int64 
 11  Count_Casualty_All               3246 non-null   int64 
dtypes: int64(7), object(5)
memory usag

In [8]:
df_factors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4799 entries, 0 to 4798
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Crash_Year                   4799 non-null   int64 
 1   Crash_Police_Region          4799 non-null   object
 2   Crash_Severity               4799 non-null   object
 3   Involving_Drink_Driving      4799 non-null   object
 4   Involving_Driver_Speed       4799 non-null   object
 5   Involving_Fatigued_Driver    4799 non-null   object
 6   Involving_Defective_Vehicle  4799 non-null   object
 7   Count_Crashes                4799 non-null   int64 
 8   Count_Fatality               4799 non-null   int64 
 9   Count_Hospitalised           4799 non-null   int64 
 10  Count_Medically_Treated      4799 non-null   int64 
 11  Count_Minor_Injury           4799 non-null   int64 
 12  Count_All_Casualties         4799 non-null   int64 
dtypes: int64(7), object(6)
memory usa

From the above, one can see that they share some columns such as:

- Crash Year
- Crash Severity
- Crash Police Region (excluding locations dataset)
- Count variables (excluding the factors dataset)

The locations and factors datasets will have to be preprocessed as their variable names do not match with the other datasets.

For the factors datasets, a column mapping was defined, and the columns were renamed based on this column mapping

In [9]:
column_mapping = {'Count_Fatality': 'Count_Casualty_Fatality',
                  'Count_Hospitalised': 'Count_Casualty_Hospitalised',
                  'Count_Medically_Treated': 'Count_Casualty_MedicallyTreated',
                  'Count_Minor_Injury': 'Count_Casualty_MinorInjury',
                  'Count_All_Casualties': 'Count_Casualty_All',
}

# renaming columns based on the dictionary mapping
df_factors_renamed = df_factors.rename(columns=column_mapping)

df_factors_renamed.columns

Index(['Crash_Year', 'Crash_Police_Region', 'Crash_Severity',
       'Involving_Drink_Driving', 'Involving_Driver_Speed',
       'Involving_Fatigued_Driver', 'Involving_Defective_Vehicle',
       'Count_Crashes', 'Count_Casualty_Fatality',
       'Count_Casualty_Hospitalised', 'Count_Casualty_MedicallyTreated',
       'Count_Casualty_MinorInjury', 'Count_Casualty_All'],
      dtype='object')

For the locations dataset, there were three different police location variables (District, Division, Region). To determine which column corresponded with the Crash_Police_Region column in the other three datasets, the unique values of these location variables were analysed.

In [10]:
# Unique district values
df_locations["Loc_Police_District"].unique()

array(['South Brisbane', 'Unknown', 'North Brisbane', 'Ipswich', 'Logan',
       'Moreton', 'Gold Coast', 'Sunshine Coast', 'Wide Bay Burnett',
       'Darling Downs', 'Capricornia', 'South West', 'Mackay',
       'Mount Isa', 'Townsville', 'Far North'], dtype=object)

In [11]:
# unique division values
df_locations["Loc_Police_Division"].unique()

array(['Acacia Ridge', 'Moorooka', 'Upper Mount Gravatt', 'Calamvale',
       'Dutton Park', 'Annerley', 'Unknown', 'Coorparoo', 'Holland Park',
       'Sherwood', 'The Gap', 'Ferny Grove', 'Indooroopilly',
       'Brisbane City', 'Carina', 'Wynnum', 'Morningside', 'Capalaba',
       'Fortitude Valley', 'West End', 'Inala', 'Mount Ommaney', 'Goodna',
       'Browns Plains', 'Sandgate', 'Carseldine',
       'Mango Hill North Lakes', 'Albany Creek', 'Petrie', 'Stafford',
       'Boondall', 'Logan Central', 'Springwood', 'Hendra', 'Dayboro',
       'Beaudesert', 'Kalbar', 'Beenleigh', 'Coomera', 'North Tamborine',
       'Caboolture', 'Burpengary', 'Deception Bay', 'Bribie Island',
       'Woodford', 'Beerwah', 'Cleveland', 'Redland Bay',
       'Russell Island', 'Macleay Island', 'Loganholme', 'Esk', 'Dunwich',
       'Karana Downs', 'Ipswich', 'Redcliffe', 'Crestmead', 'Jimboomba',
       'Yamanto', 'Booval', 'Springfield', 'Marburg', 'Rosewood',
       'Broadbeach', 'Robina', 'Southpor

In [12]:
# unique region values
df_locations["Loc_Police_Region"].unique()

array(['Brisbane', 'Unknown', 'Southern', 'South Eastern', 'North Coast',
       'Central', 'Northern', 'Far Northern'], dtype=object)

In [13]:
# unique Crash_Police_Region values in the other three datasets
print(df_driver_demographics["Crash_Police_Region"].unique())
print(df_vehicle_types["Crash_Police_Region"].unique())
print(df_factors_renamed["Crash_Police_Region"].unique())

['Brisbane' 'Central' 'Far Northern' 'North Coast' 'Northern'
 'South Eastern' 'Southern' 'Unknown']
['Brisbane' 'Central' 'Far Northern' 'North Coast' 'Northern'
 'South Eastern' 'Southern' 'Unknown']
['Brisbane' 'Central' 'Far Northern' 'North Coast' 'Northern'
 'South Eastern' 'Southern' 'Unknown']


From the above data exploration, it would seem that Loc_Police_Region matches the Crash_Police_Region column in the other three datasets.

As such, the Loc_Police_Region will be renamed to follow the naming scheme of the other datasets. Note that the Count_Casualty_Total column will also be renamed to Count_Casualty_All for the same reason.

In [14]:
df_locations_renamed = df_locations.rename(columns = {"Loc_Police_Region": "Crash_Police_Region",
                                                      "Count_Casualty_Total": "Count_Casualty_All"})

df_locations.columns

Index(['Crash_Ref_Number', 'Crash_Severity', 'Crash_Year', 'Crash_Month',
       'Crash_Day_Of_Week', 'Crash_Hour', 'Crash_Nature', 'Crash_Type',
       'Crash_Longitude', 'Crash_Latitude', 'Crash_Street',
       'Crash_Street_Intersecting', 'State_Road_Name', 'Loc_Suburb',
       'Loc_Local_Government_Area', 'Loc_Post_Code', 'Loc_Police_Division',
       'Loc_Police_District', 'Loc_Police_Region',
       'Loc_Queensland_Transport_Region', 'Loc_Main_Roads_Region',
       'Loc_ABS_Statistical_Area_2', 'Loc_ABS_Statistical_Area_3',
       'Loc_ABS_Statistical_Area_4', 'Loc_ABS_Remoteness',
       'Loc_State_Electorate', 'Loc_Federal_Electorate',
       'Crash_Controlling_Authority', 'Crash_Roadway_Feature',
       'Crash_Traffic_Control', 'Crash_Speed_Limit',
       'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition',
       'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align',
       'Crash_Road_Vert_Align', 'Crash_DCA_Code', 'Crash_DCA_Description',
       'Crash_DCA_Group_De

Now to verify that the police region and count columns match across all three datasets

In [15]:
# Get the column names for each DataFrame
columns_locations = set(df_locations_renamed.columns)
columns_driver_demographics = set(df_driver_demographics.columns)
columns_vehicle_types = set(df_vehicle_types.columns)
columns_factors = set(df_factors_renamed.columns)

# Find the shared columns using .intersection
shared_columns = list(columns_locations.intersection(columns_driver_demographics,
                                                    columns_vehicle_types,
                                                    columns_factors))


print(shared_columns)

['Crash_Year', 'Crash_Severity', 'Count_Casualty_MedicallyTreated', 'Count_Casualty_All', 'Crash_Police_Region', 'Count_Casualty_Fatality', 'Count_Casualty_Hospitalised', 'Count_Casualty_MinorInjury']


Before merging the datasets by performing a join, duplicate rows based on the datasets' shared columns should be inspected

In [16]:
# Function to calculate the number of duplicates in a DataFrame
def count_duplicates(df, key_columns):
    return df.duplicated(subset=key_columns).sum()

# Define the key columns to compare duplicates
key_columns = ['Crash_Year', 
               'Crash_Police_Region', 
               'Crash_Severity', 
               'Count_Casualty_All', 
               'Count_Casualty_Fatality', 
               'Count_Casualty_Hospitalised', 
               'Count_Casualty_MedicallyTreated', 
               'Count_Casualty_MinorInjury']

# Calculate the number of duplicates for each dataset
num_duplicates_locations = count_duplicates(df_locations_renamed, key_columns)
num_duplicates_driver = count_duplicates(df_driver_demographics, key_columns)
num_duplicates_vehicle = count_duplicates(df_vehicle_types, key_columns)
num_duplicates_factors = count_duplicates(df_factors_renamed, key_columns)

# Print the number of duplicates for each dataset
print("Number of duplicates in df_locations:", num_duplicates_locations)
print("Number of duplicates in df_driver_demographics:", num_duplicates_driver)
print("Number of duplicates in df_vehicle_types:", num_duplicates_vehicle)
print("Number of duplicates in df_factors:", num_duplicates_factors)


Number of duplicates in df_locations: 272562
Number of duplicates in df_driver_demographics: 5068
Number of duplicates in df_vehicle_types: 353
Number of duplicates in df_factors: 920


From the above, it would seem that there are a large number of duplicates for these shared columns. Because of this, performing an inner join would be computationally expensive, as the resulting merged dataframe will contain all the possible combinations of the duplicated rows.

Below is an example of an inner join between the locations and demographics datasets:

In [17]:
# Define the common key columns for merging
key_columns = ['Crash_Year', 
               'Crash_Police_Region', 
               'Crash_Severity', 
               'Count_Casualty_All',
               'Count_Casualty_Fatality', 
               'Count_Casualty_Hospitalised', 
               'Count_Casualty_MedicallyTreated', 
               'Count_Casualty_MinorInjury']

# Perform inner join on df_crash_locations and df_crash_driver_demographics
df_merged = pd.merge(df_locations_renamed, df_driver_demographics, on=key_columns, how='inner')

print(df_locations_renamed.shape)
print(df_driver_demographics.shape)
print(df_merged.shape)

(279798, 52)
(16713, 16)
(1200657, 60)


In [18]:
df_merged.columns

Index(['Crash_Ref_Number', 'Crash_Severity', 'Crash_Year', 'Crash_Month',
       'Crash_Day_Of_Week', 'Crash_Hour', 'Crash_Nature', 'Crash_Type',
       'Crash_Longitude', 'Crash_Latitude', 'Crash_Street',
       'Crash_Street_Intersecting', 'State_Road_Name', 'Loc_Suburb',
       'Loc_Local_Government_Area', 'Loc_Post_Code', 'Loc_Police_Division',
       'Loc_Police_District', 'Crash_Police_Region',
       'Loc_Queensland_Transport_Region', 'Loc_Main_Roads_Region',
       'Loc_ABS_Statistical_Area_2', 'Loc_ABS_Statistical_Area_3',
       'Loc_ABS_Statistical_Area_4', 'Loc_ABS_Remoteness',
       'Loc_State_Electorate', 'Loc_Federal_Electorate',
       'Crash_Controlling_Authority', 'Crash_Roadway_Feature',
       'Crash_Traffic_Control', 'Crash_Speed_Limit',
       'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition',
       'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align',
       'Crash_Road_Vert_Align', 'Crash_DCA_Code', 'Crash_DCA_Description',
       'Crash_DCA_Group_

In [19]:
# Perform inner join on df_merged and df_crash_vehicle_types

# Added Count_Crashes as it is a shared column with current merged dataset (locations, demographics) and vehicle types
key_columns = ['Crash_Year', 
               'Crash_Police_Region', 
               'Crash_Severity', 
               'Count_Casualty_All',
               'Count_Casualty_Fatality', 
               'Count_Casualty_Hospitalised', 
               'Count_Casualty_MedicallyTreated', 
               'Count_Casualty_MinorInjury',
               'Count_Crashes']

print(df_merged.shape)
print(df_vehicle_types.shape)

df_merged = pd.merge(df_merged, df_vehicle_types, on=key_columns, how='inner')

print(df_merged.shape)

(1200657, 60)
(3246, 12)
(658421, 63)


In [20]:
df_merged.columns

Index(['Crash_Ref_Number', 'Crash_Severity', 'Crash_Year', 'Crash_Month',
       'Crash_Day_Of_Week', 'Crash_Hour', 'Crash_Nature', 'Crash_Type',
       'Crash_Longitude', 'Crash_Latitude', 'Crash_Street',
       'Crash_Street_Intersecting', 'State_Road_Name', 'Loc_Suburb',
       'Loc_Local_Government_Area', 'Loc_Post_Code', 'Loc_Police_Division',
       'Loc_Police_District', 'Crash_Police_Region',
       'Loc_Queensland_Transport_Region', 'Loc_Main_Roads_Region',
       'Loc_ABS_Statistical_Area_2', 'Loc_ABS_Statistical_Area_3',
       'Loc_ABS_Statistical_Area_4', 'Loc_ABS_Remoteness',
       'Loc_State_Electorate', 'Loc_Federal_Electorate',
       'Crash_Controlling_Authority', 'Crash_Roadway_Feature',
       'Crash_Traffic_Control', 'Crash_Speed_Limit',
       'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition',
       'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align',
       'Crash_Road_Vert_Align', 'Crash_DCA_Code', 'Crash_DCA_Description',
       'Crash_DCA_Group_

In [21]:
# Perform inner join on df_merged and df_crash_factors
print(df_merged.shape)
print(df_factors_renamed.shape)

df_merged = pd.merge(df_merged, df_factors_renamed, on=key_columns, how='inner')

print(df_merged.shape)

(658421, 63)
(4799, 13)
(704155, 67)


In [22]:
df_merged.columns

Index(['Crash_Ref_Number', 'Crash_Severity', 'Crash_Year', 'Crash_Month',
       'Crash_Day_Of_Week', 'Crash_Hour', 'Crash_Nature', 'Crash_Type',
       'Crash_Longitude', 'Crash_Latitude', 'Crash_Street',
       'Crash_Street_Intersecting', 'State_Road_Name', 'Loc_Suburb',
       'Loc_Local_Government_Area', 'Loc_Post_Code', 'Loc_Police_Division',
       'Loc_Police_District', 'Crash_Police_Region',
       'Loc_Queensland_Transport_Region', 'Loc_Main_Roads_Region',
       'Loc_ABS_Statistical_Area_2', 'Loc_ABS_Statistical_Area_3',
       'Loc_ABS_Statistical_Area_4', 'Loc_ABS_Remoteness',
       'Loc_State_Electorate', 'Loc_Federal_Electorate',
       'Crash_Controlling_Authority', 'Crash_Roadway_Feature',
       'Crash_Traffic_Control', 'Crash_Speed_Limit',
       'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition',
       'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align',
       'Crash_Road_Vert_Align', 'Crash_DCA_Code', 'Crash_DCA_Description',
       'Crash_DCA_Group_

The resulting dataset contains over 700k rows and 67 columns. Depending on the following machine learning / deep learning models to be used, you may have to remove duplicates.

__NOTE__. Removal of duplicates results in an extremely small dataset, which does not reflect the real world data.
 
In other words, you would expect crashes to have similar characteristics most of the time, therefore duplicates make sense in this regard.

In [23]:
df_merged_deduplicated = df_merged.drop_duplicates(subset = key_columns)

print(df_merged.shape)
print(df_merged_deduplicated.shape)

(704155, 67)
(224, 67)


According to the above, there are only 224 unique instances of crashes, when the key columns are concerned. As a refresher, these were the key columns:

    Crash_Year
    Crash_Police_Region
    Crash_Severity
    Count_Casualty_All
    Count_Casualty_Fatality
    Count_Casualty_Hospitalised
    Count_Casualty_MedicallyTreated
    Count_Casualty_MinorInjury
    Count_Crashes

This suggests that there are only 224 unique combinations of the variables above.

Again, these columns were chosen because they were shared across the datasets and thus facilitated merging.

For the purposes of this notebook, only the dataset with duplicates will be considered.


# Merged Dataset Preprocessing

Currently, we are dealing with a dataset with 704155 rows and 67 columns. 

In [24]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 704155 entries, 0 to 704154
Data columns (total 67 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Crash_Ref_Number                    704155 non-null  int64  
 1   Crash_Severity                      704155 non-null  object 
 2   Crash_Year                          704155 non-null  int64  
 3   Crash_Month                         704155 non-null  object 
 4   Crash_Day_Of_Week                   704155 non-null  object 
 5   Crash_Hour                          704155 non-null  int64  
 6   Crash_Nature                        704155 non-null  object 
 7   Crash_Type                          704155 non-null  object 
 8   Crash_Longitude                     704155 non-null  float64
 9   Crash_Latitude                      704155 non-null  float64
 10  Crash_Street                        704150 non-null  object 
 11  Crash_Street_Intersecting 

Inspecting missing values

In [25]:
# Set the display option to show all rows
pd.set_option('display.max_rows', None)

# Display the entire output
print(df_merged.isna().sum())

# Reset the display option to its default value
pd.set_option('display.max_rows', 10)


Crash_Ref_Number                           0
Crash_Severity                             0
Crash_Year                                 0
Crash_Month                                0
Crash_Day_Of_Week                          0
Crash_Hour                                 0
Crash_Nature                               0
Crash_Type                                 0
Crash_Longitude                            0
Crash_Latitude                             0
Crash_Street                               5
Crash_Street_Intersecting             400034
State_Road_Name                       397594
Loc_Suburb                                 0
Loc_Local_Government_Area                  0
Loc_Post_Code                              0
Loc_Police_Division                        0
Loc_Police_District                        0
Crash_Police_Region                        0
Loc_Queensland_Transport_Region            0
Loc_Main_Roads_Region                      0
Loc_ABS_Statistical_Area_2                 0
Loc_ABS_St

We can see that a large amount of values are missing for:

    Crash_Street_Intersecting: 400034
    State_Road_Name: 397594

And to a lesser extent:

    DCA_Key_Approach_Dir: 10480

With the least amount for:

    Crash_Road_Vert_Align: 14
    Crash_DCA_Code: 16
    Crash_DCA_Description: 16


To start with, the most straightforward preprocessing step to conduct first is to simply drop the rows where Crash_Road_Vert_Align, Crash_DCA_Code, and Crash_DCA_Description have missing values.

These rows were dropped because there is not a significant amount of information lost from their removal, especially when you consider that there are over 700k rows in the dataset.

In [26]:
# Drop rows with missing values in specified columns
df_merged_nans_dropped = df_merged.dropna(subset=['Crash_Road_Vert_Align', 'Crash_DCA_Code', 'Crash_DCA_Description'])

print(df_merged.shape)
print(df_merged_nans_dropped.shape)

(704155, 67)
(704125, 67)


For the 3 variables with significant amounts of NaNs, further exploration shall be done to determine the best method to address them.

We'll start by looking at the unique values in each column

In [27]:
df_merged['Crash_Street_Intersecting'].unique()


array([nan, 'Grenadier Ct', 'Signata St', ..., 'Goorawin St',
       'Links Ave South', 'Gateway Art Rd Ramp Xi'], dtype=object)

In [28]:
df_merged_nans_dropped['State_Road_Name'].unique()

array([nan, 'Mount Lindesay Arterial Road', 'Griffith Arterial Road',
       'Cunningham Arterial Road (Ipswich Motorway)',
       'South-East Arterial Road (Pacific Motorway)',
       'Western Arterial Road (Toowong - Everton Park)',
       'Moggill Sub-Arterial Road',
       'Centenary Motorway (Ellen Grove - Toowong)',
       'Gateway Motorway (Eight Mile Plains - Nudgee)',
       'Gympie Arterial Road', 'Redcliffe Sub-Arterial Road',
       'Gateway Arterial Road (Gateway Motorway - North)',
       'East-West Arterial Road', 'Redland Sub-Arterial Road',
       'Gateway Extension Motorway (Drewvale - Eight Mile Plains)',
       'Logan Sub-Arterial Road',
       'South East Busway (Brisbane Cbd - Woolloongabba)',
       'Port Of Brisbane Road', 'Sandgate Sub-Arterial Road',
       'Albany Creek Sub-Arterial Road', 'South Pine Road',
       'Cleveland - Redland Bay Road', 'Beenleigh - Redland Bay Road',
       'Brighton - Redcliffe Road', 'Birkdale Road',
       'Brisbane - Redland Ro

In [29]:
df_merged_nans_dropped['DCA_Key_Approach_Dir'].unique()

array(['S', 'N', 'W', 'E', nan, 'U'], dtype=object)

In [30]:
# Number of unique values
print(df_merged_nans_dropped['Crash_Street_Intersecting'].nunique())
print(df_merged_nans_dropped['State_Road_Name'].nunique())
print(df_merged_nans_dropped['DCA_Key_Approach_Dir'].nunique())

6481
571
5


From the above, we can see that Crash_Street_Intersecting has over 6.4k unique values, while to a lesser extent State_Road_Name has 571, with DCA_Key_Approach_Dir having the least.

Crash_Street_Intersecting seems to contain street and road names. State_Road_Name contains road names, as well as a geographic indicator of where that road may be (Suburb, part of a highway, near a creek, etc.). DCA_Key_Approach_Dir seems to contain cardinal direction values.

Because NaN value processing can be different based on the analyses to be done, these NaNs will be left in the dataset. For example, with machine learning models like random forest, NaN values can be handled by the model inherently. On the other hand, other models might need imputation methods to handle the NaN values.

One thing to note however is that the Crash_Street_Intersecting is missing values for more than half the rows in the entire dataset. As such, it is recommended to remove this column entirely, as imputation methods may not be appropriate. If there are more missing values than available values, it does not make sense to extrapolate the former from the latter as the former could have the possibility of representing the true distribution of values.

The last step of pre-processing is to simply output to a csv file that can be used by other scripts/notebooks for analyses.

In [31]:
df_qld_crash_data = df_merged_nans_dropped
df_qld_crash_data.to_csv('qld_crash_data_merged_processed.csv', index = False)